# Advanced AD application
This notebook documents how the algorithmic (or automatic) differentiation framework may be applied to non-linear equations. For an introduction to the framework, see other tutorials on AD.

The functions in question are the normal and tangential complementary equations for contact mechanics, which are only semi-smooth (i.e. they are not differentiable everywhere):

\begin{equation}
\begin{aligned}
C_n &= \lambda_n + \text{max}(0, -\lambda_n-c_n([[u]]_n-g))\\
C_{\tau} &= \text{max}(0, b) (\lambda_{\tau}+c_{\tau}[[\dot{u}]]_{\tau})
- \text{max}(b, ||\lambda_{\tau}+c_{\tau}[[\dot{u}]]_{\tau}||)\lambda_{\tau},
\end{aligned}
\end{equation}
with $b=-F(\lambda_n+c_n([[u]]_n-g))$ and F, c, and $g$ denoting friction coefficient, numerical constants and the gap function, respectively. See [Hüeber 2008](https://elib.uni-stuttgart.de/handle/11682/4854) for a detailed derivation and discussion and [Stefansson et al. 2021](https://www.sciencedirect.com/science/article/pii/S0045782521004539) for notation.

## Implementation
The implementation is found within the `ContactMechanics` class. After defining subdomain and interface lists and ad variables, `_assign_equations` calls the methods `_contact_mechanics_normal_equation` and `_contact_mechanics_normal_equation` which compose the equations from subcomponents defined in other methods:

In [2]:
import porepy as pp
import numpy as np
import inspect

model = pp.ContactMechanics({})
print(inspect.getsource(model._assign_equations)) 

    def _assign_equations(self):
        """Assign equations to self._eq_manager.

        The ad variables are set by a previous call to _assign_ad_variables and
        accessed through self._ad.*variable_name*

        The following equations are assigned to the equation manager:
            "momentum" in the nd subdomain
            "contact_mechanics_normal" in all fracture subdomains
            "contact_mechanics_tangential" in all fracture subdomains
            "force_balance" at the matrix-fracture interfaces

        Returns
        -------
        None.

        """
        gb, Nd = self.gb, self._Nd

        g_primary: pp.Grid = gb.grids_of_dimension(Nd)[0]
        fracture_subdomains: List[pp.Grid] = gb.grids_of_dimension(Nd - 1).tolist()
        self._num_frac_cells = np.sum([g.num_cells for g in fracture_subdomains])

        matrix_fracture_interfaces = [(g_primary, g) for g in fracture_subdomains]

        # Projections between subdomains, rotations etc. must be wrapp

The simpler of the equations is defined as follows:

In [3]:
print(inspect.getsource(model._contact_mechanics_normal_equation))

    def _contact_mechanics_normal_equation(
        self,
        fracture_subdomains: List[pp.Grid],
    ) -> pp.ad.Operator:
        """
        Contact mechanics equation for the normal constraints.

        Parameters
        ----------
        fracture_subdomains : List[pp.Grid]
            List of fracture grids.

        Returns
        -------
        equation : pp.ad.Operator
            Contact mechanics equation for the normal constraints.

        """
        numerical_c_n = pp.ad.ParameterMatrix(
            self.mechanics_parameter_key,
            array_keyword="c_num_normal",
            grids=fracture_subdomains,
        )

        T_n: pp.ad.Operator = self._ad.normal_component_frac * self._ad.contact_traction

        MaxAd = pp.ad.Function(pp.ad.maximum, "max_function")
        zeros_frac = pp.ad.Array(np.zeros(self._num_frac_cells))
        u_n: pp.ad.Operator = self._ad.normal_component_frac * self._displacement_jump(
            fracture_subdomains
        )
    

## Non-smooth functions using pp.ad.Function
Handling non-smoothness in the AD setting requires the definition of extended derivatives by assigning appropriate values to the Jacobi matrices for the non-smooth function components ($\text{max}$ and $\text{abs}$) at the points in question. While this may seem somewhat technical, it is a modest price to pay for handling these equations otherwise straightforwardly using AD. We define standard Python functions and wrap them in `pp.ad.Function` returning `pp.ad.Ad_array`s having a val and a jac attribute. For instance, the maximum value function is defined and used as follows:

In [7]:
print(inspect.getsource(pp.ad.functions.maximum)) 

def maximum(
    var0: pp.ad.Ad_array, var1: Union[pp.ad.Ad_array, np.ndarray]
) -> pp.ad.Ad_array:
    """Ad maximum function represented as an Ad_array.

    The second argument is allowed to be constant, with a numpy array originally
    wrapped in a pp.ad.Array, whereas the first argument is expected to be an
    Ad_array originating from a pp.ad.Operator.

    Parameters
    ----------
    var0 : pp.ad.Ad_array
        Ad operator (variable or expression).
    var1 : Union[pp.ad.Ad_array, pp.ad.Array]
        Ad operator (variable or expression) OR ad Array.

    Returns
    -------
    pp.ad.Ad_array
        The maximum of var0 and var1 with appropriate val and jac attributes.

    """
    vals = [var0.val.copy()]
    jacs = [var0.jac.copy()]
    if isinstance(var1, np.ndarray):
        vals.append(var1.copy())
        jacs.append(sps.csr_matrix(var0.jac.shape))
    else:
        vals.append(var1.val.copy())
        jacs.append(var1.jac.copy())
    inds = vals[1] >= vals[0]

    

## Technical notes on Function wrapping
### Argument types
The wrapping of a function in the pp.ad.Function class may be slightly confusing in that the function (e.g. `pp.ad.functions.max`) takes an `Ad_array` as its argument, whereas the Function instance (e.g. `MaxAd` above) expects an `Operator`, which represents an ad variable or compound expression. The explanation lies in how the Function is *parsed* ("evaluated"), which involves the `MaxAd` asking its `_function` to operate on the values and jacobians of `var0` and `var1`, which are represented through an `Ad_array`. Puh!

### Chain rule
An ad `Funtion` is parsed as follows by `pp.ad.Operator._parse_operator`:
```
elif tree.op == Operation.evaluate:
    # This is a function, which should have at least one argument
    assert len(results) > 1
    return results[0].func(*results[1:])
```
That is, it calls the wrapped function on the ad array produced by parsing of the function argument(s). This means that the chain rule should be applied internally in the function. For a generic funtion `f` of a single variable `var` with derivative `f_prime` with respect to `var`, we have
```
def function_to_be_wrapped(var: pp.ad.Ad_array) -> pp.ad.Ad_array:
    var = f(var)
    df_dvar = f_prime(var)
    # Chain rule:
    jac = var.diagvec_mul_jac(df_dvar)
    return  pp.ad.Ad_array(var, jac)
```

### Partial functions
Some functions depend on arguments which do not have anything to do with ad. Instead of having to wrap such arguments in AD objects to be evaluated as part of parsing of the Function, one can exploit partial evaluation. For instance, the `pp.ad.functions.l2_norm` function for cell-wise vectors has been implemented for an arbitrary number of vector components. It is applied in the definition of the gap, which depends on the norm of tangential displacement jumps. The number of tangential components equals the dimension of the fracture, i.e. $nd - 1$:

In [5]:
print(inspect.getsource(model._gap))

    def _gap(
        self,
        fracture_subdomains: List[pp.Grid],
    ) -> pp.ad.Operator:
        """Gap function.

        The gap function includes an initial (constant) value and shear dilation.
        It depends linearly on the norm of tangential displacement jump:
            g = g_0 + tan(dilation_angle) * norm([[u]]_t)

        Parameters
        ----------
        fracture_subdomains : List[pp.Grid]
            List of fracture grids.

        Returns
        -------
        gap : pp.ad.Operator
            Gap function representing the distance between the fracture
            interfaces when in mechanical contact.

        """
        initial_gap: pp.ad.Operator = pp.ad.ParameterArray(
            self.mechanics_parameter_key,
            array_keyword="initial_gap",
            grids=fracture_subdomains,
        )
        angle: pp.ad.Operator = pp.ad.ParameterArray(
            self.mechanics_parameter_key,
            array_keyword="dilation_angle",
            gri